In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical  # 导入one-hot编码函数
from sklearn.model_selection import train_test_split  # 导入训练集测试集分割函数

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout  # 导入卷积层、池化层等Keras层
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM  # 导入平坦层、全连接层等Keras层  
from keras.models import Sequential, Model  # 导入Keras模型

import keras
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from keras import backend as K  # 导入Keras后端模块
import time
from sklearn.model_selection import StratifiedKFold  # 导入分层折叠交叉验证

num_classes = 4   # 类别数,即SEED4数据集是个4分类
batch_size = 128  # batch大小
img_rows, img_cols, num_chan = 8, 9, 4  # 图像行数、列数、通道数，8行9列对应电极的空间位置，通道数对应4个频带
# img_rows, img_cols, num_chan = 17, 19, 5

falx = np.load('./source/DE0.5s/t6x_89.npy')   # 加载特征数据
y = np.load('./source/DE0.5s/t6y_89_1.npy')    # 加载标签数据

# one_y = np.array([y[:1697]] * 3).reshape((-1,))
# one_y = to_categorical(one_y, num_classes)
# one_y = []
# one_y.append()

# 标签one-hot编码
one_y_1 = np.array(y.reshape(-1,))
one_y_1 = to_categorical(one_y_1, num_classes)


acc_list = []  # 存储每折accuracy
std_list = []  # 存储每折std
all_acc = []   # 存储每轮次5折accuracy
for nb in range(15):    # 训练15轮，对应15个被试
    K.clear_session()   # 清空上一次训练的模型图
    start = time.time()
    
    # 获取本轮次训练数据
    one_falx_1 = falx[nb * 3:nb * 3 + 3]
    one_falx_1 = one_falx_1.reshape((-1, 6, img_rows, img_cols, 5))

    # ###============= random select ============####
    # permutation = np.random.permutation(one_y_1.shape[0])
    # one_falx_2 = one_falx_1[permutation, :]
    # one_falx = one_falx_2[0:3400]
    # one_y_2 = one_y_1[permutation, :]
    # one_y = one_y_2[0:3400]
    # ###============= random select ============####
    one_y = one_y_1
    
    # 取特征数据第2-5通道作为模型输入
    one_falx = one_falx_1[:,:,:,:,1:5]

    print(one_y.shape)
    print(one_falx.shape)
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
    
    # 5折交叉验证
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []


    # create model
    for train, test in kfold.split(one_falx, one_y.argmax(1)):
        img_size = (img_rows, img_cols, num_chan)

        # 定义基础卷积网络，CNN
        def create_base_network(input_dim):

            seq = Sequential()
            seq.add(Conv2D(64, 5, activation='relu', padding='same', name='conv1', input_shape=input_dim))
            seq.add(Conv2D(128, 4, activation='relu', padding='same', name='conv2'))
            seq.add(Conv2D(256, 4, activation='relu', padding='same', name='conv3'))
            seq.add(Conv2D(64, 1, activation='relu', padding='same', name='conv4'))
            seq.add(MaxPooling2D(2, 2, name='pool1'))
            seq.add(Flatten(name='fla1'))
            seq.add(Dense(512, activation='relu', name='dense1'))
            seq.add(Reshape((1, 512), name='reshape'))
            return seq



        # 初始化CNN
        base_network = create_base_network(img_size)
        # 定义LSTM的输入
        input_1 = Input(shape=img_size)
        input_2 = Input(shape=img_size)
        input_3 = Input(shape=img_size)
        input_4 = Input(shape=img_size)
        input_5 = Input(shape=img_size)
        input_6 = Input(shape=img_size)

        # out_all是CNN的输出结果，
        out_all = Concatenate(axis=1)([base_network(input_1), base_network(input_2), base_network(input_3), base_network(input_4), base_network(input_5), base_network(input_6)])
        
        # 初始化lstm层
        lstm_layer = LSTM(128, name='lstm')(out_all)
        
        #  out_layer是LSTM的输出结果
        out_layer = Dense(4, activation='softmax', name='out')(lstm_layer)
        
        # 初始化完整的模型(CNN+LSTM)
        model = Model([input_1, input_2, input_3, input_4, input_5, input_6], out_layer)
        # model.summary()
        # Compile model
        # 定义模型的参数,包括优化器等
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.adam_v2.Adam(),
                      metrics=['accuracy'])
        # Fit the model
        x_train = one_falx[train]
        y_train = one_y[train]
        model.fit([x_train[:, 0], x_train[:, 1], x_train[:, 2], x_train[:, 3], x_train[:, 4], x_train[:, 5]], y_train, epochs=100, batch_size=128, verbose=1)
        # evaluate the model
        x_test = one_falx[test]
        y_test = one_y[test]
        scores = model.evaluate([x_test[:, 0], x_test[:, 1], x_test[:, 2], x_test[:, 3], x_test[:, 4], x_test[:, 5]], y_test, verbose=0)

        print("%.2f%%" % (scores[1] * 100)) # 输出准确率
        all_acc.append(scores[1] * 100)

    # 输出5折平均accuracy和std    
    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("进度： {}".format(nb))
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))   # 本轮次训练时间
    
# 输出15轮次平均accuracy和std
# Acc_mean就是最终结果
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print("Acc_mean: {}".format(np.mean(acc_list)))
print("Std_all: {}".format(np.std(std_list)))

(3303, 4)
(3303, 6, 8, 9, 4)


2023-11-09 13:08:22.923176: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-09 13:08:22.929794: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/conda/miniconda3/envs/czq2/lib/python3.10/site-packages/torch/lib/:/home/user/conda/miniconda3/envs/czq2/lib/python3.10/site-packages/torch/lib/:/home/user/conda/miniconda3/envs/czq2/lib/python3.10/site-packages/torch/lib/:/home/user/conda/miniconda3/envs/czq2/lib/python3.10/site-packages/torch/lib/::/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2023-11-09 13:08:22.930499: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mention

Epoch 1/100
21/21 [==============================] - 13s 498ms/step - loss: 1.0415 - accuracy: 0.5371
Epoch 2/100
21/21 [==============================] - 10s 493ms/step - loss: 0.9927 - accuracy: 0.5674
Epoch 3/100
21/21 [==============================] - 10s 493ms/step - loss: 0.9864 - accuracy: 0.5674
Epoch 4/100
21/21 [==============================] - 10s 487ms/step - loss: 0.9775 - accuracy: 0.5670
Epoch 5/100
21/21 [==============================] - 10s 497ms/step - loss: 0.9758 - accuracy: 0.5651
Epoch 6/100
21/21 [==============================] - 10s 496ms/step - loss: 0.9613 - accuracy: 0.5651
Epoch 7/100
21/21 [==============================] - 10s 485ms/step - loss: 0.9572 - accuracy: 0.5651
Epoch 8/100
21/21 [==============================] - 10s 494ms/step - loss: 0.9183 - accuracy: 0.5727
Epoch 9/100
21/21 [==============================] - 10s 493ms/step - loss: 0.9264 - accuracy: 0.5757
Epoch 10/100
21/21 [==============================] - 10s 493ms/step - loss: 0.886

21/21 [==============================] - 10s 494ms/step - loss: 0.1359 - accuracy: 0.9565
Epoch 81/100
21/21 [==============================] - 10s 495ms/step - loss: 0.1467 - accuracy: 0.9440
Epoch 82/100
21/21 [==============================] - 10s 489ms/step - loss: 0.0949 - accuracy: 0.9739
Epoch 83/100
21/21 [==============================] - 10s 490ms/step - loss: 0.1006 - accuracy: 0.9682
Epoch 84/100
21/21 [==============================] - 10s 490ms/step - loss: 0.0960 - accuracy: 0.9701
Epoch 85/100
21/21 [==============================] - 10s 487ms/step - loss: 0.0555 - accuracy: 0.9883
Epoch 86/100
21/21 [==============================] - 10s 490ms/step - loss: 0.0887 - accuracy: 0.9709
Epoch 87/100
21/21 [==============================] - 10s 486ms/step - loss: 0.0887 - accuracy: 0.9724
Epoch 88/100
21/21 [==============================] - 10s 493ms/step - loss: 0.0493 - accuracy: 0.9917
Epoch 89/100
21/21 [==============================] - 10s 487ms/step - loss: 0.0421 - 